In [ ]:
# El dataset cuenta con estas images

<userid> <pose> <expression> <eyes> <scale>.pgm
<userid> is the user id of the person in the image, and this field has 20 values: an2i, at33, boland, bpm, ch4f, cheyer, choon, danieln, glickman, karyadi, kawamura, kk49, megak, mitchell, night, phoebe, saavik, steffi, sz24, and tammo.
<pose> is the head position of the person, and this field has 4 values: straight, left, right, up.
<expression> is the facial expression of the person, and this field has 4 values: neutral, happy, sad, angry.
<eyes> is the eye state of the person, and this field has 2 values: open, sunglasses.
<scale> is the scale of the image, and this field has 3 values: 1, 2, and 4. 1 indicates a full-resolution image (128 columns by 120 rows); 2 indicates a half-resolution image (64 by 60); 4 indicates a quarter-resolution image (32 by 30).
If you've been looking closely in the image directories, you may notice that some images have a .bad suffix rather than the .pgm suffix. As it turns out, 16 of the 640 images taken have glitches due to problems with the camera setup; these are the .bad images. Some people had more glitches than others, but everyone who got ``faced'' should have at least 28 good face images (out of the 32 variations possible, discounting scale).

 **Nombre de los integrantes:**


* **Integrante 1:** Juan José Monsalve Patiño \\
* **Integrante 2:** Pamela Escobar Palacios \\
* **Integrante 3:** José Julián Aguirre Ramírez \\
* **Integrante 4:** Santiago Mejia Carmona


<center>
    <img src="https://blogs.elespectador.com/wp-content/uploads/2017/09/logo-Universidad-Nacional.png" width="500" alt="logo" />
</center>

**Definición del problema**

DEFINIR EL PROBLEMA ---- 

## Instalar librerias necesarias

In [1]:
# @title
from keras.utils import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import os
import seaborn as sns
from skimage import feature, io
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
import random
import itertools
from shutil import copyfile, rmtree
from pathlib import Path
from glob import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow_hub as hub
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
import shutil
from google.colab import drive
drive.mount('/content/drive')
from sklearn.metrics import confusion_matrix


Mounted at /content/drive


# Análisis descriptivo y exploratorio:

In [2]:
sin_gafas = '/content/drive/MyDrive/imagenes_analitica/con_gafas'
con_gafas = '/content/drive/MyDrive/imagenes_analitica/sin_gafas'
print('Se cuentan con', len(os.listdir(sin_gafas)), 'sin gafas')
print('--------------------------------------------------------------')
print('Se cuentan con', len(os.listdir(con_gafas)), 'con gafas')

Se cuentan con 0 sin gafas
--------------------------------------------------------------
Se cuentan con 0 con gafas


#Cargar y Preprocesar Imágenes


In [4]:
import os
from skimage import io, color, transform
import numpy as np
from sklearn.model_selection import train_test_split

# Directorio que contiene las carpetas con gafas y sin gafas
dataset_dir = '/content/drive/MyDrive/imagenes_analitica'

# Cargar imágenes y etiquetas
X = []
y = []

for label, category in enumerate(['con_gafas', 'sin_gafas']):
    category_dir = os.path.join(dataset_dir, category)
    for filename in os.listdir(category_dir):
        img_path = os.path.join(category_dir, filename)
        img = io.imread(img_path)
        img = transform.resize(img, (64, 64, 3)).flatten()  # Ajusta el tamaño según sea necesario
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

 # Construir y Entrenar el Modelo

In [7]:
from tensorflow.keras import layers, models

# las imágenes son en escala de grises y aplanadas (64x64x3)
model = models.Sequential([
    layers.Dense(4096, activation='relu', input_shape=(64*64*3,)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
16/16 [==============================] - 17s 1s/step - loss: 11.6016 - accuracy: 0.4689 - val_loss: 6.1543 - val_accuracy: 0.4320
Epoch 2/10
16/16 [==============================] - 16s 1s/step - loss: 2.1524 - accuracy: 0.5651 - val_loss: 1.5810 - val_accuracy: 0.5760
Epoch 3/10
16/16 [==============================] - 16s 1s/step - loss: 1.1804 - accuracy: 0.5972 - val_loss: 0.5913 - val_accuracy: 0.7280
Epoch 4/10
16/16 [==============================] - 16s 1s/step - loss: 0.7365 - accuracy: 0.6593 - val_loss: 0.5155 - val_accuracy: 0.7920
Epoch 5/10
16/16 [==============================] - 15s 979ms/step - loss: 0.5913 - accuracy: 0.7154 - val_loss: 0.4852 - val_accuracy: 0.7760
Epoch 6/10
16/16 [==============================] - 14s 897ms/step - loss: 0.6079 - accuracy: 0.6954 - val_loss: 0.6030 - val_accuracy: 0.6720
Epoch 7/10
16/16 [==============================] - 15s 888ms/step - loss: 0.6152 - accuracy: 0.6834 - val_loss: 0.5899 - val_accuracy: 0.7280
Epoch 8/10

# Pruebas del Modelo


In [9]:
# Suponiendo que ya has entrenado tu modelo y tienes X_test y y_test listos.

# Hacer predicciones en el conjunto de prueba
predictions = model.predict(X_test)

# Convertir las predicciones a clases binarias (0 o 1) según un umbral (por ejemplo, 0.5)
threshold = 0.5
predicted_classes = (predictions > threshold).astype(int)

# Evaluar el rendimiento del modelo
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predicted_classes)
report = classification_report(y_test, predicted_classes)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)


4/4 [==============================] - 0s 90ms/step
Accuracy: 0.824
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.82      0.84        71
           1       0.78      0.83      0.80        54

    accuracy                           0.82       125
   macro avg       0.82      0.83      0.82       125
weighted avg       0.83      0.82      0.82       125

